In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools
import cufflinks as cf
import envelope_plot as ep
from ipywidgets import interact, interactive
from IPython.display import display
import ipywidgets as widgets
import os
import math

cf.go_offline()
plotly.offline.init_notebook_mode(connected=True)

def extract_keys(my_dict, key_list):
    return {k: my_dict[k] for k in key_list}
    

# Prepare data

Every domain instance requires the optimal path length which is
read in via a json results file in the same format as the experiment results.

Specify the paths to the results yielding information on the optimal plans below.
Then specify paths to the experiment results file. A dataframe will be constructed
using this information


In [ ]:
# optimal_plans = ['../results/astar_minima1500.json','../results/astar_orz100d.json',
#                  '../results/astar_korf4real.json','../results/astar_hansen-bigger-d-wide3_no-startup.json']
optimal_plans = [
    '../results/dataFRESH_BENCHMARK_OPTIMAL-09-35-20-07-19.json',
    '../results/astar_korf4real.json'
]

experiments = [
    '../results/dataCPU_PERCENTILE-20-00-01-08-19.json.gz'
]

experiment_data = ep.analyze_results(optimal_plans, experiments)

# experiment_data[
#     experiment_data['success'] == False
# ].algorithmName

# experiment_data['withinOpt']


In [ ]:
# Within Optimal plot data prep

domains = {
    'Minima 1500x1500':'minima1500',
    'Uniform 1500x1500':'uniform1500',
    'Dragon Age: Origins orz':'orz100d',
    'Dragon Age: Origins ost':'ost000a',
    'Rooms: 8':'8room',
    'Rooms: 16':'16room',
    'Starcraft: The Frozen Sea':'TheFrozenSea',
    'Starcraft: Cauldron':'Cauldron',
    'Sliding Tile - Korf 4x4':'tiles/korf/4/real',
    'Racetrack (Hansen, 3-wide)': 'hansen-bigger-d-wide3'
}

master_config = {
    'weight': 'W',
    'lookaheadStrategy': 'Look',
    'envelopeSearchStrategy': 'Env-L',
    'generatePredecessors': 'Pred'
}

just_weight = extract_keys(master_config, ['weight'])
wLookaheadConfig = extract_keys(master_config, ['weight', 'lookaheadStrategy'])
biEsConfig = extract_keys(master_config, [
    'weight', 'lookaheadStrategy', 'envelopeSearchStrategy', 'generatePredecessors'
])

groupings = {
    'LSS_LRTA_STAR':{},
    'BI_ES': biEsConfig,
    'TIME_BOUNDED_A_STAR': wLookaheadConfig
}

bi_lookahead_options = {
    'Greedy Env, Greedy Frontier':0,
    'Greedy Env, A* Frontier': 1,
    'A* Env, Greedy Frontier': 2,
    'A* Env, A* Frontier': 3,
    'Greedy Env, Pseudo F Frontier': 4,
    'A* Env, Psuedo F Frontier': 5
}

expansion_data, cpu_data = ep.prepare_within_opt_plots(experiment_data,domains.values(), groupings,
                                                       percentile=99)

# Plot data

Please use the drop down menus to select the domain instance and the plot type

In [ ]:
def plot_within_opt(domain, display_type, result_type,
                    algorithms, weights, bi_lookahead, generatePredecessors,
                    xaxis_name, yaxis_name):
    
    # Don't modify original, so copy the data
    data_copy = None
    xaxis_config = None
    if result_type == 'CPU':
        data_copy = cpu_data.get(domain,{}).copy()
        xaxis_config = cpu_data['xaxis']
    elif result_type == 'Expansion':
        data_copy = expansion_data.get(domain,{}).copy()
        xaxis_config = expansion_data['xaxis']
    
    if len(algorithms) == 0 or len(data_copy) == 0:
        return None, None, None
    
    
    trimmed_df = pd.DataFrame({}, columns=data_copy.columns)
    
    for alg in algorithms:
        alg_data = data_copy[data_copy.algCode == alg]
        
        # Filters
    
        # Generic filter loop - if param exists and values are specified, filter by it
    
        alg_filters = groupings[alg]
        for group, values in zip(
            ['weight'],
            [weights]
        ):
            if group in alg_filters and len(values) > 0:
                alg_data = alg_data[alg_data[group].isnull() | alg_data[group].isin(values)]

            
        # Special Bi-ES: filter by lookahead strategy and generate predecessors
        if alg == 'BI_ES':
            alg_data = alg_data[alg_data.generatePredecessors == generatePredecessors]
            
            if 0 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 1 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 2 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            if 3 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            if 4 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'PSEUDO_F') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 5 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'PSEUDO_F') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            
        
        trimmed_df = trimmed_df.append(alg_data)
        
        
    if display_type == 'Table':
        table = ff.create_table(trimmed_df)
        table.layout.update({'title':domain})
        
        return (table, None, None)
    elif display_type == 'Plot':
        # cufflinks doesn't support error bars...? At least I can't find it
        # creating plotly plot by hand instead... ugh!
        
        palette = cf.colors.get_scales('spectral', trimmed_df.algorithmName.nunique())
        num_colors = len(palette)
        data_comp = []
        for idx, alg_name in enumerate(trimmed_df.algorithmName.unique()):
            alg_df = trimmed_df[trimmed_df.algorithmName == alg_name]
            # get error bounds
            error_y = None
            error_x = None
            if 'yLbound' in alg_df.columns:
                error_y = dict(
                    type='data',
                    symmetric=False,
                    array=alg_df.yLbound,
                    arrayminus=alg_df.yRbound,
                    visible=True
                )
            if 'xLbound' in alg_df.columns:
                error_x = dict(
                    type='data',
                    symmetric=False,
                    array=alg_df.xLbound,
                    arrayminus=alg_df.xRbound,
                    visible=True
                )
                
            
            data_comp.append(go.Scatter(
                x = alg_df[xaxis_config['data']],
                y = alg_df.withinOpt,
                xaxis = xaxis_name,
                yaxis = yaxis_name,
                name = alg_name,
                mode = 'lines+markers',
                error_y = error_y,
                error_x = error_x,
                line = dict(
                    color=palette[idx % num_colors]
                )
            ))
        
        xaxis_layout = dict(
            title=xaxis_config['title'],
            type='log',
            anchor=yaxis_name
        )
        yaxis_layout = dict(
            title='Factor of Optimal',
            type='log',
            anchor=xaxis_name
        )
        
        return data_comp, xaxis_layout, yaxis_layout
    else:
        return (None, None, None)


def interactive_plot(domain, display_type, result_type, algorithms, weights, bi_lookahead, generatePredecessors):
    data_comp, xaxis, yaxis = plot_within_opt(domain, display_type, result_type, algorithms,
                               weights, bi_lookahead, generatePredecessors,
                               'x', 'y')
    
    if data_comp != None:
        fig_comp = go.Figure(data=data_comp, layout=go.Layout(title=domain, xaxis=xaxis, yaxis=yaxis))
        plotly.iplot(fig_comp, filename='jupyter-within-opt-plot')
    
    
plot_widget = interactive(
    interactive_plot,
    domain=domains,
    display_type=['Plot','Table'],
    algorithms=widgets.SelectMultiple(
        options={
            'TBA*':'TIME_BOUNDED_A_STAR',
            'LSS-LRTA*':'LSS_LRTA_STAR',
            'Bidirectional Envelope': 'BI_ES'
        },
        description='Algorithms (Select multiple)',
        value=[]
    ),
    result_type=['CPU','Expansion'],
    weights=widgets.SelectMultiple(
        options=list(filter(lambda x: x==x, experiment_data.weight.unique())), # remove NaN
        description='Weights (Select multiple, or none for all)',
        value=[]
    ),
    bi_lookahead=widgets.SelectMultiple(
        options=bi_lookahead_options,
        description='Bi-ES lookahead strategies',
        value=[2]
    ),
    generatePredecessors=False
)

display(plot_widget)


# Button code

handler_output = widgets.Output()
def click_handler(btn):
    with handler_output: # for debug printing
        args = plot_widget.kwargs.copy()
        
        # create master plot. Use 3 columns, calculate rows
        rows = int(math.ceil(len(domains) / 2.0))
        fig = go.Figure(layout=go.Layout(
            title='All plots',
            height=rows * 1000,
            width=1800,
            annotations=[]
        ))
        
        relative_height = 1.0 / float(rows)
        
        trace_list = list()
        row = 0
        col = 0
        for idx, key_value in enumerate(domains.items()):
            domain = key_value[1]
            args['domain'] = domain
            
            xaxis_key = ''
            yaxis_key = ''
            if idx == 0:
                args['xaxis_name'] = 'x'
                args['yaxis_name'] = 'y'
                xaxis_key = 'xaxis'
                yaxis_key = 'yaxis'
            else:
                idx_str = str(idx+1)
                
                args['xaxis_name'] = 'x' + idx_str
                args['yaxis_name'] = 'y' + idx_str
                xaxis_key = 'xaxis' + idx_str
                yaxis_key = 'yaxis' + idx_str
            
            data, xaxis, yaxis = plot_within_opt(**args)
            
            if idx != 0:
                for scatter in data:
                    scatter['showlegend'] = False
            
            # positioning
            x_start = .5 * col
            x_end = x_start + .5
            xaxis.update({'domain': [x_start + 0.05, x_end - 0.05]})
            
            y_start = relative_height * row
            y_end = y_start + relative_height
            yaxis.update({'domain': [y_start + 0.07, y_end - 0.07]})
            
            fig.layout[xaxis_key] = xaxis
            fig.layout[yaxis_key] = yaxis
            
            fig.layout.annotations = fig.layout.annotations + (dict(
                showarrow=False,
                text=domain,
                xref='paper',
                yref='paper',
                x=x_start + 0.2,
                y=y_end - 0.02,
                font={'size':16}
            ),)
            
            trace_list.extend(data)
            
            col = col + 1
            if col % 2 == 0:
                col = 0
                row = row + 1
            
        fig.add_traces(trace_list)
        
#         plotly.iplot(fig, filename='master-plot')
        pio.write_image(fig, '../output/plots.pdf')
           
            
        
button = widgets.Button(
    tooltip='Plot all domains using the current configuration',
    description='Plot All Domains',
    disabled=False,
    button_style='success'
)
button.on_click(click_handler)

display(button, handler_output)
        

In [ ]:
# test = experiment_data[(experiment_data['algorithmName'] == 'BI_ES') &
#                        (experiment_data['domainName'] == 'RACETRACK') &
#                        (experiment_data['actionDuration'] == 10)
#                       ]

# resetCount = 0
# index = 0

# for idx, row in test.iterrows():
#     if 'envelopeResets' in row.attributes:
#         resetCount += row.attributes['envelopeResets']
#         index += 1
    
# avg = resetCount / index
# print(avg)

print(cf.colors.get_scales('spectral', 5))